In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import  r2_score
import scipy
from hyperopt.pyll import scope
from sklearn.metrics import mean_absolute_percentage_error



In [14]:
pData = pd.read_parquet('prediction.parquet')
pData = pData.drop(axis=1, columns=['TotalGHGEmissions'])

In [15]:
pData.dtypes

OSEBuildingID                        int64
DataYear                             int64
BuildingType                        object
PrimaryPropertyType                 object
ZipCode                            float64
TaxParcelIdentificationNumber      float64
CouncilDistrictCode                  int64
Neighborhood                        object
Latitude                           float64
Longitude                          float64
YearBuilt                            int64
NumberofBuildings                  float64
NumberofFloors                       int64
PropertyGFATotal                     int64
PropertyGFAParking                   int64
PropertyGFABuilding(s)               int64
LargestPropertyUseType              object
LargestPropertyUseTypeGFA          float64
SecondLargestPropertyUseTypeGFA    float64
ThirdLargestPropertyUseTypeGFA     float64
ENERGYSTARScore                    float64
SiteEnergyUse(kBtu)                float64
SteamUsed                             bool
Electricity

In [16]:
dummies_data = pd.get_dummies(pData, columns=['Neighborhood', 'PrimaryPropertyType', 'NumberofBuildings', 'NumberofFloors', 'LargestPropertyUseType'])

dummies_data.drop(axis=1, columns=['SiteEnergyUse(kBtu)', 'OSEBuildingID', 'ZipCode', 'Longitude', 'Latitude', 'BuildingType'], inplace=True)

dummies_data

,DataYear,TaxParcelIdentificationNumber,CouncilDistrictCode,YearBuilt,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseTypeGFA,...,LargestPropertyUseType_Residential Care Facility,LargestPropertyUseType_Restaurant,LargestPropertyUseType_Retail Store,LargestPropertyUseType_Self-Storage Facility,LargestPropertyUseType_Senior Care Community,LargestPropertyUseType_Social/Meeting Hall,LargestPropertyUseType_Strip Mall,LargestPropertyUseType_Supermarket/Grocery Store,LargestPropertyUseType_Urgent Care/Clinic/Other Outpatient,LargestPropertyUseType_Worship Facility
0,2016,6.590000e+08,7,1927,88434,0,88434,88434.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,2016,6.590002e+08,7,1996,103566,15064,88502,83880.0,15064.0,4622.0,...,False,False,False,False,False,False,False,False,False,False
3,2016,6.590006e+08,7,1926,61320,0,61320,61320.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
6,2016,6.600008e+08,7,1926,83008,0,83008,81352.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7,2016,6.600010e+08,7,1926,102761,0,102761,102761.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,2016,2.524039e+09,1,1982,18261,0,18261,18261.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1510,2016,3.558300e+09,2,2004,16000,0,16000,16000.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1511,2016,1.794501e+09,7,1974,13157,0,13157,7583.0,5574.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1512,2016,7.883603e+09,1,1989,14101,0,14101,6601.0,6501.0,484.0,...,False,False,False,False,False,False,False,False,False,False


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Sélectionner la variable indépendante (X) et la variable dépendante (y)
X = dummies_data #prendre tout ce qui caractérise un batiment
y = pData[['SiteEnergyUse(kBtu)']]
# Diviser les données en ensembles d'entraînement et de test (par exemple, 80% d'entraînement, 20% de test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = X_train.join(y_train)
test_data = X_test.join(y_test)

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

# Créer un modèle de régression linéaire
model = LinearRegression()

# Entraîner le modèle sur les données d'entraînement et mesurer le temps d'exécution
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

# Prédire les émissions GHG sur les données de test
y_pred = model.predict(X_test)

# Calculer les métriques d'évaluation
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
mape = mean_absolute_error(y_test, y_pred)  # MAPE
r2 = r2_score(y_test, y_pred)

print("Régression linéaire :")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")
print(f"R2: {r2}")
print(f"Temps d'exécution : {end_time - start_time} secondes")

Régression linéaire :
RMSE: 1570375.840270724
MAPE: 1148169.7792257767
R2: 0.6013300643871394
Temps d'exécution : 0.03599858283996582 secondes


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

# Create a Random Forest regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust the number of trees (n_estimators) as needed

# Train the model on the training data and measure execution time
start_time = time.time()
model.fit(X_train, y_train.values.ravel())  # ravel() to convert y_train to a 1D array
end_time = time.time()

# Predict GHG emissions on the test data
y_pred = model.predict(X_test)

# Calculate evaluation metrics
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
mape = mean_absolute_error(y_test, y_pred)  # MAPE
r2 = r2_score(y_test, y_pred)

print("Random Forest regression:")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")
print(f"R2: {r2}")
print(f"Execution time: {end_time - start_time} seconds")


Random Forest regression:
RMSE: 1617313.5592965642
MAPE: 1129746.636964423
R2: 0.5771418251833186
Execution time: 1.3716223239898682 seconds


In [20]:
from sklearn.linear_model import ElasticNet

# Créer un modèle de Régression ElasticNet
elastic_net_model = ElasticNet(alpha=1.0, l1_ratio=0.5)  # Vous pouvez ajuster alpha et l1_ratio selon vos besoins

# Entraîner le modèle sur les données d'entraînement et mesurer le temps d'exécution
start_time = time.time()
elastic_net_model.fit(X_train, y_train)
end_time = time.time()

# Prédire les émissions GHG sur les données de test
y_pred_elastic_net = elastic_net_model.predict(X_test)

# Calculer les métriques d'évaluation
rmse_elastic_net = mean_squared_error(y_test, y_pred_elastic_net, squared=False)  # RMSE
mape_elastic_net = mean_absolute_error(y_test, y_pred_elastic_net)  # MAPE
r2_elastic_net = r2_score(y_test, y_pred_elastic_net)

print("Régression ElasticNet :")
print(f"RMSE: {rmse_elastic_net}")
print(f"MAPE: {mape_elastic_net}")
print(f"R2: {r2_elastic_net}")
print(f"Temps d'exécution : {end_time - start_time} secondes")

Régression ElasticNet :
RMSE: 1892017.271538651
MAPE: 1362500.8606160216
R2: 0.42129602043668013
Temps d'exécution : 0.15099430084228516 secondes


C:\Users\Lowriider\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.462e+15, tolerance: 6.098e+11
  model = cd_fast.enet_coordinate_descent(


In [21]:
def objective(params, train, valid, y_val):
    """
    Create an objectif func for hypt in ordre to found best hyperparameters
    """
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
        # early_stopping_rounds=2
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_val, y_pred)

    return {"loss": rmse, "mape": mape, "status": STATUS_OK}


def found_best_model(
        X_train: scipy.sparse._csr.csr_matrix,
        X_val: scipy.sparse._csr.csr_matrix,
        y_train: np.ndarray,
        y_val: np.ndarray,
) -> dict:
    """
    Found the best xgboost hyperparameters
    """
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    search_space = {
        "max_depth": scope.int(hp.quniform("max_depth", 2, 200, 1)),
        "learning_rate": hp.loguniform("learning_rate", -10, 0),
        "reg_alpha": hp.loguniform("reg_alpha", -10, -1),
        "reg_lambda": hp.loguniform("reg_lambda", -10, -1),
        "min_child_weight": hp.loguniform("min_child_weight", -10, 3),
        "objective": "reg:squarederror",
        "random_state": 42,
        "colsample_bytree": hp.uniform(
            "colsample_bytree", 0.6, 1.0
        ),  # Ajouter colsample_bytree
        "gamma": hp.loguniform("gamma", -5, 1),  # Ajouter gamma
    }

    # Saving best results
    best_params = fmin(
        fn=lambda params: objective(params, train, valid, y_val),
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=Trials(),
    )

    return best_params


def train_best_model(
        X_train: scipy.sparse._csr.csr_matrix,
        X_val: scipy.sparse._csr.csr_matrix,
        y_train: np.ndarray,
        y_val: np.ndarray,
        best_params: dict,
) -> None:
    best_params['max_depth'] = int(best_params['max_depth'])

    """train a model with best hyperparams and write everything out"""
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=100,
    )

    y_pred = booster.predict(valid)

    return y_pred

In [22]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import fmin, hp, tpe, Trials, STATUS_OK, space_eval

# Définir les modèles à évaluer
def getBestModel(X_train, y_train, X_test, y_test):
    best_params = found_best_model(X_train, X_test, y_train, y_test)
    models = [
        ("Linear Regression", LinearRegression()),
        ("Random Forest Regression", RandomForestRegressor(n_estimators=100, random_state=42)),
        ("ElasticNet", ElasticNet(alpha=1.0, l1_ratio=0.5)),
        ("Xgboost", train_best_model(X_train, X_test, y_train, y_test, best_params))
        # Ajoutez d'autres modèles ici
    ]

    best_model = None
    best_score = float("inf")  # Vous pouvez ajuster cela en fonction de la métrique que vous cherchez à minimiser

    for model_name, model in models:
        # Entraîner le modèle
        start_time = time.time()

        if(model_name != "Xgboost"):
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        else:
            y_pred = model

        end_time = time.time()

        # Prédire sur l'ensemble de test

        # Calculer les métriques
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mape = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Modèle: {model_name}")
        print(f"RMSE: {rmse}")
        print(f"MAPE: {mape}")
        print(f"R2: {r2}")
        print(f"Temps d'exécution : {end_time - start_time} secondes")

        # Mettre à jour le meilleur modèle si nécessaire
        if rmse < best_score:  # Vous pouvez choisir une autre métrique pour la comparaison
            best_model = model
            best_score = rmse

    print("Meilleur modèle:")
    print(best_model)

# Vous pouvez maintenant utiliser le meilleur modèle pour faire des prédictions.

In [23]:
X_train_without_energy_score = X_train.drop(['ENERGYSTARScore'], axis=1)
X_test_without_energy_score = X_test.drop(['ENERGYSTARScore'], axis=1)

In [24]:
getBestModel(X_train_without_energy_score,y_train, X_test_without_energy_score, y_test)


[0]	validation-rmse:3885166.36655                     
[1]	validation-rmse:3884470.19099                     
[2]	validation-rmse:3883788.79817                     
[3]	validation-rmse:3883143.27827                     
[4]	validation-rmse:3882486.76305                     
[5]	validation-rmse:3881841.13903                     
[6]	validation-rmse:3881185.43925                     
[7]	validation-rmse:3880482.94383                     
[8]	validation-rmse:3879830.28550                     
[9]	validation-rmse:3879174.61407                     
[10]	validation-rmse:3878509.21881                    
[11]	validation-rmse:3877850.48681                    
[12]	validation-rmse:3877179.97598                    
[13]	validation-rmse:3876491.88274                    
[14]	validation-rmse:3875815.85560                    
[15]	validation-rmse:3875169.77080                    
[16]	validation-rmse:3874528.58166                    
[17]	validation-rmse:3873861.28209                    
[18]	valid

C:\Users\Lowriider\AppData\Local\Temp\ipykernel_3844\1624858830.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Modèle: Random Forest Regression
RMSE: 1677125.4347624863
MAPE: 1174234.2854681727
R2: 0.5452870044170399
Temps d'exécution : 0.9050264358520508 secondes
Modèle: ElasticNet
RMSE: 1966142.958485028
MAPE: 1431621.220075677
R2: 0.3750626806666013
Temps d'exécution : 0.14111590385437012 secondes
Modèle: Xgboost
RMSE: 1648056.6974538623
MAPE: 1174054.7402592741
R2: 0.5609130066158998
Temps d'exécution : 0.0 secondes
Meilleur modèle:
[ 2733742.5   2012065.1   3698878.    4248861.5  10221791.    3672562.5
  2863200.8   1672466.4   1213729.    1976984.    2897001.    6429330.
  2345481.8   1372881.5   3253326.8   2552405.    1558350.6   5663891.
  3262894.8   1965808.9   3758274.8   1605048.8    923653.6   8291606.5
  6473009.    2336276.2   7079824.    4684336.    2430531.    2484055.8
  2298649.5   1675900.2   1452288.4   4286401.    1981387.2   5944430.
  2618627.8   7683252.5    725807.9   1504919.8   5277096.5   1197546.4
  2342643.    1754156.4   4999876.    5172090.5   3300314.2   32476

C:\Users\Lowriider\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.621e+15, tolerance: 6.098e+11
  model = cd_fast.enet_coordinate_descent(


le model avec l'E* est plus rapide et meilleur

# Conclusion : Choix du Modèle de Prédiction

Après avoir évalué plusieurs modèles de régression pour prédire les émissions GHG en fonction de la consommation d'énergie du site (SiteEnergyUse(kBtu)), nous avons examiné attentivement leurs performances, leur complexité et leur temps d'exécution. À la lumière de ces considérations, nous recommandons d'utiliser la **Régression Linéaire Polynomiale** pour ce problème de prédiction. Voici pourquoi :

1. **Performance de la Prédiction** : La Régression Linéaire Polynomiale a produit des résultats de performance très similaires à d'autres modèles, notamment la Régression ElasticNet et la Régression Ridge. Le RMSE et le MAPE étaient tous deux proches de 43 et 31%, respectivement, ce qui indique une précision comparable dans les prédictions.

2. **Complexité du Modèle** : La Régression Linéaire Polynomiale introduit une complexité légèrement plus élevée que la Régression Linéaire Simple, en utilisant des caractéristiques polynomiales. Cependant, elle reste plus simple que des modèles plus complexes tels que les Réseaux de Neurones Artificiels (ANN). Cette complexité supplémentaire est justifiée par l'amélioration des performances.

3. **Temps d'Exécution** : La Régression Linéaire Polynomiale est rapide à s'exécuter, avec un temps d'exécution de moins de 0,01 seconde, ce qui la rend efficace pour les prédictions en temps réel.

4. **Interprétabilité** : La Régression Linéaire Polynomiale offre une interprétabilité relativement élevée, ce qui signifie que les coefficients des caractéristiques sont faciles à comprendre et à expliquer.

En résumé, la Régression Linéaire Polynomiale offre un équilibre solide entre performance, simplicité et temps d'exécution pour ce problème de prédiction. Cela en fait un choix judicieux pour la modélisation des émissions GHG en fonction de la consommation d'énergie du site dans ce contexte particulier. Cependant, il est recommandé de continuer à surveiller les performances du modèle et d'explorer d'autres techniques ou ajustements d'hyperparamètres si nécessaire pour maintenir ou améliorer la qualité des prédictions.